In [187]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, mean_absolute_error,mean_squared_error,classification_report
from sklearn.linear_model import LinearRegression, LogisticRegression;
import numpy as np

In [188]:
train = pd.read_csv("/home/ganso133/uni/DAA/TP-DAA/training_data.csv")
test = pd.read_csv("/home/ganso133/uni/DAA/TP-DAA/test_data.csv")

In [189]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   city_name           5000 non-null   object 
 1   magnitude_of_delay  5000 non-null   object 
 2   delay_in_seconds    5000 non-null   int64  
 3   affected_roads      4915 non-null   object 
 4   record_date         5000 non-null   object 
 5   luminosity          5000 non-null   object 
 6   avg_temperature     5000 non-null   float64
 7   avg_atm_pressure    5000 non-null   float64
 8   avg_humidity        5000 non-null   float64
 9   avg_wind_speed      5000 non-null   float64
 10  avg_precipitation   5000 non-null   float64
 11  avg_rain            5000 non-null   object 
 12  incidents           5000 non-null   object 
dtypes: float64(5), int64(1), object(7)
memory usage: 507.9+ KB


In [190]:
print(train['city_name'].value_counts())
#Useless values
train.head()

Guimaraes    5000
Name: city_name, dtype: int64


,city_name,magnitude_of_delay,delay_in_seconds,affected_roads,record_date,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_precipitation,avg_rain,incidents
0,Guimaraes,UNDEFINED,0,",",2021-03-15 23:00,DARK,12.0,1013.0,70.0,1.0,0.0,Sem Chuva,None
1,Guimaraes,UNDEFINED,385,"N101,",2021-12-25 18:00,DARK,12.0,1007.0,91.0,1.0,0.0,Sem Chuva,None
2,Guimaraes,UNDEFINED,69,",",2021-03-12 15:00,LIGHT,14.0,1025.0,64.0,0.0,0.0,Sem Chuva,Low
3,Guimaraes,MAJOR,2297,"N101,R206,N105,N101,N101,N101,N101,N101,N101,N...",2021-09-29 09:00,LIGHT,15.0,1028.0,75.0,1.0,0.0,Sem Chuva,Very_High
4,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101,",2021-06-13 11:00,LIGHT,27.0,1020.0,52.0,1.0,0.0,Sem Chuva,High


In [191]:
print(train['incidents'].value_counts())

None         2028
High         1073
Low           718
Very_High     603
Medium        578
Name: incidents, dtype: int64


In [192]:
# Atribute an order so its easier to understand
replace_map = {'incidents':{'None':0, 'Low':1, 'Medium':2,'High':3,'Very_High':4}}


train.replace(replace_map,inplace=True)
#train.head()

In [193]:
# Here we clearly need to clarify that Major > moderate but undefined values need investigation
print(train['magnitude_of_delay'].value_counts())



UNDEFINED    4314
MAJOR         678
MODERATE        8
Name: magnitude_of_delay, dtype: int64


In [194]:
print(train['avg_rain'].value_counts())

Sem Chuva         4756
chuva fraca        214
chuva moderada      27
chuva forte          3
Name: avg_rain, dtype: int64


In [195]:
print(train['luminosity'].value_counts())

DARK         2489
LIGHT        2325
LOW_LIGHT     186
Name: luminosity, dtype: int64


In [196]:
# Atribute an order so its easier to understand
replace_map1 = {'avg_rain':{'Sem Chuva':0, 'chuva fraca':1, 'chuva moderada':2,'chuva forte':3}}
replace_map2 = {'luminosity': {'DARK':0,'LOW_LIGHT':1,'LIGHT':2,}}


train.replace(replace_map1,inplace=True)
test.replace(replace_map1,inplace=True)
train.replace(replace_map2,inplace=True)
test.replace(replace_map2,inplace=True)
train.head()

,city_name,magnitude_of_delay,delay_in_seconds,affected_roads,record_date,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_precipitation,avg_rain,incidents
0,Guimaraes,UNDEFINED,0,",",2021-03-15 23:00,0,12.0,1013.0,70.0,1.0,0.0,0,0
1,Guimaraes,UNDEFINED,385,"N101,",2021-12-25 18:00,0,12.0,1007.0,91.0,1.0,0.0,0,0
2,Guimaraes,UNDEFINED,69,",",2021-03-12 15:00,2,14.0,1025.0,64.0,0.0,0.0,0,1
3,Guimaraes,MAJOR,2297,"N101,R206,N105,N101,N101,N101,N101,N101,N101,N...",2021-09-29 09:00,2,15.0,1028.0,75.0,1.0,0.0,0,4
4,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101,",2021-06-13 11:00,2,27.0,1020.0,52.0,1.0,0.0,0,3


In [197]:
train['record_date'] = pd.to_datetime(train['record_date'],format='%Y-%m-%d %H:%M',errors='coerce')
assert train['record_date'].isnull().sum()==0
train['record_date_month'] = train['record_date'].dt.month
train['record_date_day'] = train['record_date'].dt.day
train['record_date_isWeekend'] = train['record_date'].dt.dayofweek
train['record_date_hour'] = train['record_date'].dt.hour

replace_map3 = {0:0,1:0,2:0,3:0,4:0,5:1,6:1}
train['record_date_isWeekend'].replace(replace_map3,inplace=True)


test['record_date'] = pd.to_datetime(test['record_date'],format='%Y-%m-%d %H:%M',errors='coerce')
assert test['record_date'].isnull().sum()==0
test['record_date_month'] = test['record_date'].dt.month
test['record_date_day'] = test['record_date'].dt.day
test['record_date_isWeekend'] = test['record_date'].dt.dayofweek
test['record_date_hour'] = test['record_date'].dt.hour

test['record_date_isWeekend'].replace(replace_map3,inplace=True)


In [198]:
train["affected_roads"].value_counts()

N101,N101,N101,N101,N101                                                                                                    1514
N101,N101,N101,N101,N101,                                                                                                   1078
,                                                                                                                            709
N101,N101                                                                                                                    149
N101,N101,N101,N101,N101,N101                                                                                                114
                                                                                                                            ... 
N101,N101,N105,N101,N101,N101,N101,N105,N101,N101,N101,N309,N101,N101,N101,N105                                                1
N101,N101,N309,N309,N309,N309,N101,N101,N105,N101,N101,N309,N309,N309,N101,N101,N105,N105,N101,N1

In [199]:
train.head()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   city_name              5000 non-null   object        
 1   magnitude_of_delay     5000 non-null   object        
 2   delay_in_seconds       5000 non-null   int64         
 3   affected_roads         4915 non-null   object        
 4   record_date            5000 non-null   datetime64[ns]
 5   luminosity             5000 non-null   int64         
 6   avg_temperature        5000 non-null   float64       
 7   avg_atm_pressure       5000 non-null   float64       
 8   avg_humidity           5000 non-null   float64       
 9   avg_wind_speed         5000 non-null   float64       
 10  avg_precipitation      5000 non-null   float64       
 11  avg_rain               5000 non-null   int64         
 12  incidents              5000 non-null   int64         
 13  rec

In [200]:
x = train[["delay_in_seconds","avg_temperature","avg_atm_pressure","avg_humidity","avg_wind_speed","avg_rain","record_date_month","record_date_day","record_date_isWeekend","record_date_hour"]]
y = train[["incidents"]]
x_train, x_test,y_train, y_test = train_test_split(x,np.ravel(y),test_size=0.3,random_state=101023)


In [201]:
# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 100,random_state=13122001) 
 
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(x_train, y_train)
 
# performing predictions on the test dataset
y_pred = clf.predict(x_test)

print("ACCURACY OF THE MODEL: ", sk.metrics.accuracy_score(y_test, y_pred))

ACCURACY OF THE MODEL:  0.9186666666666666


## Submission Tests


In [202]:
x_train = train[["delay_in_seconds","avg_temperature","avg_atm_pressure","avg_humidity","avg_wind_speed","avg_rain","record_date_month","record_date_day","record_date_isWeekend","record_date_hour"]]
y_train = train[["incidents"]]

x_test = test[["delay_in_seconds","avg_temperature","avg_atm_pressure","avg_humidity","avg_wind_speed","avg_rain","record_date_month","record_date_day","record_date_isWeekend","record_date_hour"]]





In [203]:
# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 100,random_state=13122001) 
 
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(x_train,y_train)
 
# performing predictions on the test dataset
y_pred = clf.predict(x_test)

#print("ACCURACY OF THE MODEL: ", sk.metrics.accuracy_score(y_test, y_pred))

/home/ganso133/anaconda3/envs/env/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [204]:
f = open("myfile.csv", "w")

replace_map = {0:'None', 1:'Low', 2:'Medium',3:'High',4:'Very_High'}



f.write("RowId,Incidents\n")

for i in range(y_pred.size):
    f.write(str(i+1))
    f.write(",")
    f.write(replace_map[y_pred[i]])
    f.write("\n")